Project 1  Rubric.   Boot Camp: Data Visualization and Data Analytics 
Question 4 Analysis

Source Data file: 
1.'clean_states.csv', a subset of ‘USA Real Estate Datatest’ from Kaggle.com https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset
Assumptions: 1. clean data file may contain duplicates. 
2.uscities.csv from UnitedStatesCitiesDB on kaggle.com for coordinates of Latitude and Longitude of cities. 

4 selected states: Connecticut, Massachusetts, New Jersey, New York. 
Average Price and Standard Deviation (std) for each State reveals NY with the highest variance in STD, and Mean being houses with 3 Bedrooms and 2 Bathrooms.  

The Latitude and Longitude coordinates for all 274 cities in NY State were not available in the uscities.csv. 106 of 274 were available.  I think the data would be more meaningful if we could see the difference in the average price by city over a period of time, perhaps by including the Price and the Sold Date. 

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
import os
import geopandas as gpd
import hvplot.pandas


In [2]:
#read csv file
states_df=pd.read_csv("clean_states.csv")
states_df

,Unnamed: 0,price,bed,bath,acre_lot,house_size,state,city
0,27821,579900.0,4.0,3.0,2.34,3320.0,Connecticut,Coventry
1,27826,215000.0,2.0,1.0,0.91,960.0,Connecticut,East Windsor
2,27828,144900.0,2.0,1.0,0.36,860.0,Connecticut,Vernon
3,27829,174900.0,3.0,2.0,0.19,1280.0,Connecticut,Manchester
4,27830,289900.0,4.0,2.0,0.38,1617.0,Connecticut,East Windsor
...,...,...,...,...,...,...,...,...
185883,733133,475000.0,4.0,3.0,0.23,2252.0,New Jersey,South Brunswick
185884,733136,350000.0,2.0,2.0,0.01,1551.0,New Jersey,Piscataway
185885,733146,241000.0,2.0,2.0,0.50,1077.0,New Jersey,North Brunswick
185886,733147,629000.0,5.0,2.0,0.06,2084.0,New Jersey,New Brunswick


In [3]:
#remove the column 'Unamed:0'
states_df = states_df[['price', 'bed', 'bath','acre_lot','house_size','state','city']].reset_index(drop=True)
states_df

,price,bed,bath,acre_lot,house_size,state,city
0,579900.0,4.0,3.0,2.34,3320.0,Connecticut,Coventry
1,215000.0,2.0,1.0,0.91,960.0,Connecticut,East Windsor
2,144900.0,2.0,1.0,0.36,860.0,Connecticut,Vernon
3,174900.0,3.0,2.0,0.19,1280.0,Connecticut,Manchester
4,289900.0,4.0,2.0,0.38,1617.0,Connecticut,East Windsor
...,...,...,...,...,...,...,...
185883,475000.0,4.0,3.0,0.23,2252.0,New Jersey,South Brunswick
185884,350000.0,2.0,2.0,0.01,1551.0,New Jersey,Piscataway
185885,241000.0,2.0,2.0,0.50,1077.0,New Jersey,North Brunswick
185886,629000.0,5.0,2.0,0.06,2084.0,New Jersey,New Brunswick


In [4]:
#find the Average/ Mean of house prices, and the Standard Deviation. 
aggregate_df= states_df.groupby("state").agg(["mean","std"]).round()

C:\Users\nreetz\AppData\Local\Temp\ipykernel_21284\437001107.py:2: FutureWarning: ['city'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  aggregate_df= states_df.groupby("state").agg(["mean","std"]).round()


Std for State of New York has the highest number compared to Connecticut, Massachusetts, and New Jersey
The standard deviation measures how far apart the numbers are from the mean of the data set.
Interestingly, the average number of bedrooms is 3 with 2 bathrooms

In [5]:
#Get the New York state Data in a dataframe ny_cities_df
ny_cities_df = states_df[states_df['state'] == 'New York']
ny_cities_df

,price,bed,bath,acre_lot,house_size,state,city
49777,225000.0,4.0,2.0,0.24,1239.0,New York,Copake
49778,40000.0,2.0,1.0,0.20,690.0,New York,Nassau
49779,125000.0,3.0,2.0,1.00,980.0,New York,Hoosick Falls
49780,299900.0,3.0,1.0,0.32,1608.0,New York,Niverville
49781,379500.0,4.0,3.0,0.28,2080.0,New York,Philmont
...,...,...,...,...,...,...,...
63457,1750000.0,2.0,1.0,0.41,1001.0,New York,New York City
63458,1650000.0,1.0,1.0,0.24,718.0,New York,New York City
63459,1591380.0,1.0,2.0,0.05,883.0,New York,New York City
63460,2250000.0,2.0,3.0,0.05,1252.0,New York,New York City


In [6]:
#Dataframe 'aggcity_df' has the average price of houses for each of the 274 cities in the ny_cities_df. 
aggcity_df = ny_cities_df.groupby('city').agg(["mean","std"])
aggcity_df

C:\Users\nreetz\AppData\Local\Temp\ipykernel_21284\4154986263.py:2: FutureWarning: ['state'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  aggcity_df = ny_cities_df.groupby('city').agg(["mean","std"])


price                      bed                bath  \
                   mean            std      mean       std      mean   
city                                                                   
Airmont    8.304462e+05   47309.239950  4.692308  0.470679  3.000000   
Amenia     3.185000e+05   38518.227453  3.000000  0.000000  2.000000   
Arden      4.250000e+05       0.000000  4.000000  0.000000  3.000000   
Ardsley    9.125909e+05  314599.164333  3.500000  0.597614  2.590909   
Argyle     2.266500e+05   63918.961355  3.833333  1.599210  2.333333   
...                 ...            ...       ...       ...       ...   
Witherbee  5.900000e+04       0.000000  3.000000  0.000000  1.000000   
Woodhaven  9.136667e+05  253468.098433  5.296296  1.324759  2.592593   
Woodside   1.187170e+06  362825.122363  4.176471  1.674579  2.220588   
Yonkers    5.907300e+05  203367.743443  3.426987  1.325230  2.162662   
Yulan      4.250000e+05       0.000000  3.000000  0.000000  3.000000   

                     acre_lot             house_size              
                std      mean       std         mean         std  
city                                                              
Airmont    0.748331  0.701923  0.119901  2363.000000  526.457216  
Amenia     0.000000  0.643571  0.362780  1819.714286  173.037759  
Arden      0.000000  0.040000  0.000000  1376.000000    0.000000  
Ardsley    1.181568  0.328182  0.122887  2242.454545  592.153192  
Argyle     0.758098  0.655000  0.197375  2182.833333  555.042843  
...             ...       ...       ...          ...         ...  
Witherbee  0.000000  0.400000  0.000000  1425.000000    0.000000  
Woodhaven  0.930643  0.061852  0.026752  1860.777778  613.830303  
Woodside   0.417726  0.053235  0.007618  1831.073529  261.994036  
Yonkers    0.900294  0.129113  0.150616  1667.536044  628.660373  
Yulan      0.000000  0.650000  0.000000  1480.000000    0.000000  

[274 rows x 10 columns]

In [7]:
aggcity_df.columns

MultiIndex([(     'price', 'mean'),
            (     'price',  'std'),
            (       'bed', 'mean'),
            (       'bed',  'std'),
            (      'bath', 'mean'),
            (      'bath',  'std'),
            (  'acre_lot', 'mean'),
            (  'acre_lot',  'std'),
            ('house_size', 'mean'),
            ('house_size',  'std')],
           )

In [8]:
# read source data file uscities.csv for coordinates of latitude and Longitude of US Cities. 
citieslatlng_df = pd.read_csv("uscities.csv")
citieslatlng_df.columns


Index(['city', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id'],
      dtype='object')

In [9]:
citieslatlng_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28333,Gross,Gross,NE,Nebraska,31015,Boyd,42.9461,-98.5697,2,6,polygon,False,True,America/Chicago,3,68719,1840011032
28334,Lotsee,Lotsee,OK,Oklahoma,40143,Tulsa,36.1334,-96.2091,2,39,polygon,False,True,America/Chicago,3,74063,1840021674
28335,The Ranch,The Ranch,MN,Minnesota,27087,Mahnomen,47.3198,-95.6952,2,2,polygon,False,True,America/Chicago,3,56557,1840039629
28336,Shamrock,Shamrock,OK,Oklahoma,40037,Creek,35.9113,-96.5772,2,2,polygon,False,True,America/Chicago,3,74068,1840022701


In [10]:
nycitieslatlng_df=citieslatlng_df[(citieslatlng_df['state_name']=='New York')]
nycitieslatlng_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
18,Brooklyn,Brooklyn,NY,New York,36047,Kings,40.6501,-73.9496,2559903,14221,polygon,False,True,America/New_York,1,11212 11213 11216 11233 11238 11209 11214 1122...,1840034030
19,Queens,Queens,NY,New York,36081,Queens,40.7498,-73.7976,2230722,7885,polygon,False,True,America/New_York,1,11361 11362 11363 11364 11354 11355 11356 1135...,1840034002
34,Manhattan,Manhattan,NY,New York,36061,New York,40.7834,-73.9662,1628706,27545,polygon,False,True,America/New_York,1,10026 10027 10030 10037 10039 10001 10011 1001...,1840034000
39,Bronx,Bronx,NY,New York,36005,Bronx,40.8501,-73.8662,1418207,12862,polygon,False,True,America/New_York,1,10453 10457 10460 10458 10467 10468 10451 1045...,1840033999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27635,Byersville,Byersville,NY,New York,36051,Livingston,42.5839,-77.7914,43,32,polygon,False,False,America/New_York,3,14517,1840024065
27687,Saltaire,Saltaire,NY,New York,36103,Suffolk,40.6388,-73.1949,41,68,polygon,False,True,America/New_York,3,11706,1840005135
27896,Thendara,Thendara,NY,New York,36043,Herkimer,43.7010,-74.9971,30,20,polygon,False,False,America/New_York,3,13472,1840033970
27901,Thousand Island Park,Thousand Island Park,NY,New York,36045,Jefferson,44.2888,-76.0260,30,39,polygon,False,False,America/New_York,3,13692,1840023988


In [11]:
#drop unneeded columns in the nycitieslatlng_df
nycitieslatlng_df = nycitieslatlng_df[['state_name', 'city', 'lat', 'lng']]
nycitieslatlng_df

,state_name,city,lat,lng
0,New York,New York,40.6943,-73.9249
18,New York,Brooklyn,40.6501,-73.9496
19,New York,Queens,40.7498,-73.7976
34,New York,Manhattan,40.7834,-73.9662
39,New York,Bronx,40.8501,-73.8662
...,...,...,...,...
27635,New York,Byersville,42.5839,-77.7914
27687,New York,Saltaire,40.6388,-73.1949
27896,New York,Thendara,43.7010,-74.9971
27901,New York,Thousand Island Park,44.2888,-76.0260


In [12]:
print(nycitieslatlng_df.count())
print(aggcity_df.index)
print(aggcity_df.index.name)

state_name    880
city          880
lat           880
lng           880
dtype: int64
Index(['Airmont', 'Amenia', 'Arden', 'Ardsley', 'Argyle', 'Arverne', 'Astoria',
       'Austerlitz', 'Bardonia', 'Barryville',
       ...
       'White Plains', 'Whitehall', 'Whitestone', 'Willsboro', 'Wingdale',
       'Witherbee', 'Woodhaven', 'Woodside', 'Yonkers', 'Yulan'],
      dtype='object', name='city', length=274)
city


In [13]:
merge_df = pd.DataFrame
merge_df = aggcity_df.join(nycitieslatlng_df.set_index('city'), how='left')
merge_df

C:\Users\nreetz\AppData\Local\Temp\ipykernel_21284\2157458562.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  merge_df = aggcity_df.join(nycitieslatlng_df.set_index('city'), how='left')


,"(price, mean)","(price, std)","(bed, mean)","(bed, std)","(bath, mean)","(bath, std)","(acre_lot, mean)","(acre_lot, std)","(house_size, mean)","(house_size, std)",state_name,lat,lng
city,,,,,,,,,,,,,
Airmont,8.304462e+05,47309.239950,4.692308,0.470679,3.000000,0.748331,0.701923,0.119901,2363.000000,526.457216,New York,41.0992,-74.0990
Amenia,3.185000e+05,38518.227453,3.000000,0.000000,2.000000,0.000000,0.643571,0.362780,1819.714286,173.037759,NaN,NaN,NaN
Arden,4.250000e+05,0.000000,4.000000,0.000000,3.000000,0.000000,0.040000,0.000000,1376.000000,0.000000,NaN,NaN,NaN
Ardsley,9.125909e+05,314599.164333,3.500000,0.597614,2.590909,1.181568,0.328182,0.122887,2242.454545,592.153192,New York,41.0135,-73.8395
Argyle,2.266500e+05,63918.961355,3.833333,1.599210,2.333333,0.758098,0.655000,0.197375,2182.833333,555.042843,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Witherbee,5.900000e+04,0.000000,3.000000,0.000000,1.000000,0.000000,0.400000,0.000000,1425.000000,0.000000,New York,44.0837,-73.5371
Woodhaven,9.136667e+05,253468.098433,5.296296,1.324759,2.592593,0.930643,0.061852,0.026752,1860.777778,613.830303,NaN,NaN,NaN
Woodside,1.187170e+06,362825.122363,4.176471,1.674579,2.220588,0.417726,0.053235,0.007618,1831.073529,261.994036,NaN,NaN,NaN


In [14]:
#view the list of cities in the file.  See that the NaN values are there because the city is not in the uscities file so no Latitude and Longitude data for that city.  Only 106 of 274 cities have 
# the latitude and longitude data. 
clist = nycitieslatlng_df['city'].unique()
clist.sort()
print(clist)

['Accord' 'Adams Center' 'Airmont' 'Akron' 'Albany' 'Albertson' 'Albion'
 'Alexandria Bay' 'Altamont' 'Altmar' 'Amagansett' 'Ames' 'Amityville'
 'Amsterdam' 'Angola' 'Angola on the Lake' 'Apalachin' 'Aquebogue'
 'Ardsley' 'Arkport' 'Arlington' 'Armonk' 'Asharoken' 'Atlantic Beach'
 'Au Sable Forks' 'Auburn' 'Averill Park' 'Baiting Hollow' 'Baldwin'
 'Baldwin Harbor' 'Baldwinsville' 'Ballston Spa' 'Balmville' 'Bardonia'
 'Barnum Island' 'Batavia' 'Baxter Estates' 'Bay Park' 'Bay Shore'
 'Bayport' 'Bayville' 'Baywood' 'Beacon' 'Beaver Dam Lake' 'Bedford Hills'
 'Belle Terre' 'Bellerose' 'Bellerose Terrace' 'Belleville' 'Bellmore'
 'Bellport' 'Belmont' 'Bemus Point' 'Bethpage' 'Billington Heights'
 'Binghamton' 'Binghamton University' 'Black River' 'Blasdell' 'Blauvelt'
 'Bliss' 'Blodgett Mills' 'Bloomfield' 'Bloomingburg' 'Bloomville'
 'Blue Point' 'Bohemia' 'Bolton Landing' 'Brasher Falls' 'Breesport'
 'Brentwood' 'Brewerton' 'Brewster' 'Brewster Hill' 'Briarcliff Manor'
 'Bridgehampton

In [15]:
new_df = pd.merge(ny_cities_df, nycitieslatlng_df, on='city')
new_df.head()

,price,bed,bath,acre_lot,house_size,state,city,state_name,lat,lng
0,125000.0,3.0,2.0,1.00,980.0,New York,Hoosick Falls,New York,42.9009,-73.35
1,49000.0,3.0,1.0,0.39,1161.0,New York,Hoosick Falls,New York,42.9009,-73.35
2,122750.0,4.0,2.0,0.13,1978.0,New York,Hoosick Falls,New York,42.9009,-73.35
3,109000.0,4.0,3.0,0.05,1890.0,New York,Hoosick Falls,New York,42.9009,-73.35
4,129900.0,2.0,2.0,0.10,1120.0,New York,Hoosick Falls,New York,42.9009,-73.35


In [16]:
city_group_df = new_df.groupby('city').mean()
city_group_df

C:\Users\nreetz\AppData\Local\Temp\ipykernel_21284\1194382996.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  city_group_df = new_df.groupby('city').mean()


,price,bed,bath,acre_lot,house_size,lat,lng
city,,,,,,,
Airmont,830446.153846,4.692308,3.000000,0.701923,2363.000000,41.0992,-74.0990
Ardsley,912590.909091,3.500000,2.590909,0.328182,2242.454545,41.0135,-73.8395
Bardonia,561486.363636,2.954545,2.727273,0.244545,2044.727273,41.1129,-73.9823
Blauvelt,804200.000000,4.000000,2.400000,0.738000,2435.600000,41.0689,-73.9545
Brewster,444891.625000,2.791667,2.333333,0.520000,2165.208333,41.3969,-73.6151
...,...,...,...,...,...,...,...
West Haverstraw,436999.500000,3.500000,2.500000,0.180000,1721.500000,41.2063,-73.9883
West Nyack,717666.666667,4.333333,2.333333,0.350000,2142.000000,41.0907,-73.9714
White Plains,574168.936937,3.018018,2.144144,0.307297,1733.513514,41.0220,-73.7549


In [17]:
city_plot = city_group_df.hvplot.points(
    'lng',
    'lat',
    size='price',
    scale = .01,
    geo = True,
    tiles ='OSM',
    frame_width = 600,
    frame_height = 400,
    hover_cols = ['city', 'bed', 'bath', 'acre_lot', 'house_size']
)

city_plot

c:\Users\nreetz\AppData\Local\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lng,lat]   (price,city,bed,bath,acre_lot,house_size)